# Hemocyte Recruitment Analysis - Preprocessing 
This notebook takes abdomen and hemocyte segmented features extracted by QuPath and consolidates features from all samples into dataframes for downstream analysis. Further, sample names are mapped and XY coordinates of hemocytes are scaled relative to the abdomen centroid. 

## Import required packages

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import path
from pathlib import Path
import glob
sns.set_style('white')

## Set up input/output directories and experiment info

In [8]:

base_path = Path("/Users/sbandya/Desktop/hemocyte_recruitment_image_quantification/")
sample_list = pd.read_csv(base_path.joinpath("sample_list.csv"),sep=",",header=0)
sample_mappings = pd.read_csv(base_path.joinpath("sample_mapping.tsv"),sep="\t",header=0)

# Paths to abdomen measurements and hemocyte detection measurements
am_path= base_path.joinpath('data','midsections','annotation_measurements')
dm_path= base_path.joinpath('data','midsections','detection_measurements')

outputs = base_path.joinpath('results','midsections')




## Create a dataframe of abdomen features

In [9]:

def get_sample_condition(imagename,sample_mapping):
    sampleid = imagename[:2]
    condition = sample_mapping[sample_mapping['Sample #'] == int(sampleid)]
    return condition['Condition'].iloc[0]

all_annotation_measurement_files = glob.glob(path.join(am_path , "*.txt"))
am_list = []
for amf in all_annotation_measurement_files:
    df = pd.read_csv(amf, index_col=None, header=0, sep="\t")
    am_list.append(df)


abdomen_coordinates = pd.concat(am_list, axis=0, ignore_index=True)

## annotate each fly abdomen with experimental group info 
abdomen_coordinates['condition'] = abdomen_coordinates['Image'].apply(get_sample_condition,sample_mapping=sample_list)
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'C', 'Fed', abdomen_coordinates['condition'])
abdomen_coordinates['condition'] = np.where(abdomen_coordinates['condition'] == 'S', 'Starved', abdomen_coordinates['condition'])

abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid X µm', 'Centroid_X')
abdomen_coordinates.columns = abdomen_coordinates.columns.str.replace('Centroid Y µm', 'Centroid_Y')
print(abdomen_coordinates.shape)
abdomen_coordinates[:5]


(40, 14)


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,condition
0,75C_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.41,350.34,38,38,100,210.03,180922.8,1648.0,Fed
1,60E_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,407.74,330.97,36,36,100,168.72,213374.3,1926.7,Fed
2,24E_F3_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,383.65,360.84,87,87,100,366.05,237669.9,1998.3,Fed
3,76B_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,404.40,297.69,115,115,100,723.23,159008.5,1629.7,Starved
4,76A_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,391.34,316.31,111,111,100,645.43,171978.8,1608.8,Starved


## Map Fly and Section IDs for each abdomen image

In [10]:
# update sample names in the image column to new names in the "fly_section" format 
def map_samples(imageid, sample_mapping_lookup):
    try:
        fly_section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0]
        fly_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[0]
        section_id = sample_mapping_lookup[sample_mapping_lookup['old']==imageid]['new'].iloc[0].split("_")[1]
    except: 
        fly_section_id = "notfound"
        fly_id = "none"
        section_id = "none"
    return(pd.Series([fly_section_id,fly_id,section_id]))

abdomen_coordinates[['fly_section_ID','fly_ID','section_ID']] = abdomen_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

abdomen_coordinates

,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Num Detections,Num Positive,Positive %,Num Positive per mm^2,Area µm^2,Perimeter µm,condition,fly_section_ID,fly_ID,section_ID
0,75C_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,441.41,350.34,38,38,100,210.03,180922.8,1648.0,Fed,75_17,75,17
1,60E_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,407.74,330.97,36,36,100,168.72,213374.3,1926.7,Fed,60_22,60,22
2,24E_F3_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,383.65,360.84,87,87,100,366.05,237669.9,1998.3,Fed,1_2,1,2
3,76B_1_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,404.40,297.69,115,115,100,723.23,159008.5,1629.7,Starved,76_7,76,7
4,76A_6_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,391.34,316.31,111,111,100,645.43,171978.8,1608.8,Starved,76_6,76,6
5,86D_3_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,431.44,318.57,87,87,100,553.93,157060.0,1624.2,Starved,86_18,86,18
6,84E_3_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,434.36,296.36,28,28,100,192.18,145698.9,1450.7,Starved,84_23,84,23
7,84E_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,423.51,324.95,11,11,100,70.33,156404.9,1545.2,Starved,84_22,84,22
8,72C_5_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,412.65,318.85,117,117,100,438.76,266658.4,2083.3,Fed,72_15,72,15
9,52G_2_FITC.tif,PathAnnotationObject,NaN,Image,Polygon,444.47,325.45,14,14,100,65.61,213389.7,1791.2,Fed,52_32,52,32


## Create a dataframe of hemocyte features from all samples - scale coordinates relative to abdomen centroid and map fly and section IDs 

In [11]:
from scipy.spatial import distance
import math 
sns.set_palette("pastel")

def zero_scale_x(image, x, polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0] 
    zero_scaled_x = x-polygon_x
    return(zero_scaled_x)

def zero_scale_y(image, y, polygon):
   polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0] 
   zero_scaled_y = y-polygon_y
   return(zero_scaled_y)

def get_centroid_distance(image,x,y,polygon):
    polygon_x = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_X'].iloc[0]
    polygon_y = abdomen_coordinates[abdomen_coordinates['Image']==image]['Centroid_Y'].iloc[0]
    centroid_distance = distance.euclidean([polygon_x, polygon_y], [x,y])

    return centroid_distance
    

all_detection_measurement_files = glob.glob(path.join(dm_path , "*.txt"))
dm_list = []

for dm in all_detection_measurement_files:
    #print(dm)
    detection_measurements = pd.read_csv(dm,sep="\t",header=0)
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid X µm', 'Centroid_X')
    detection_measurements.columns = detection_measurements.columns.str.replace('Centroid Y µm', 'Centroid_Y')

    detection_measurements["centroid_x_zeroscaled"] = detection_measurements.apply(
        lambda x: zero_scale_x(x.Image, x.Centroid_X,polygon=abdomen_coordinates), axis=1) 
    detection_measurements["centroid_y_zeroscaled"] = detection_measurements.apply(
        lambda x: zero_scale_y(x.Image, x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    detection_measurements["centroid_distances"] = detection_measurements.apply(
        lambda x: get_centroid_distance(x.Image, x.Centroid_X,x.Centroid_Y,polygon=abdomen_coordinates), axis=1)
    
    dm_list.append(detection_measurements)
    
    
#print(dm_list)
hemocyte_coordinates = pd.concat(dm_list, axis=0, ignore_index=True)
hemocyte_coordinates[['fly_section_ID','fly_ID','section_ID']] = hemocyte_coordinates.apply(lambda x: map_samples(x.Image, sample_mapping_lookup=sample_mappings), axis=1) 

hemocyte_coordinates[:5]

,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Nucleus: Area,Nucleus: Perimeter,Nucleus: Circularity,...,Delaunay: Max distance,Delaunay: Min distance,Delaunay: Mean triangle area,Delaunay: Max triangle area,centroid_x_zeroscaled,centroid_y_zeroscaled,centroid_distances,fly_section_ID,fly_ID,section_ID
0,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,431.16,152.82,5.4900,8.0218,1.0000,...,351.0382,15.9458,2145.0256,5813.8784,-10.25,-197.52,197.785775,75_17,75,17
1,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,444.58,156.58,11.3722,14.1223,0.7165,...,75.4955,4.9826,155.4801,572.5013,3.17,-193.76,193.785930,75_17,75,17
2,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,452.20,156.48,19.9993,20.4010,0.6038,...,14.9418,4.5568,11.4363,22.9957,10.79,-193.86,194.160047,75_17,75,17
3,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,465.84,157.54,14.1172,16.4980,0.6518,...,45.5685,3.4697,26.0180,64.5575,24.43,-192.80,194.341619,75_17,75,17
4,75C_6_FITC.tif,Positive,Positive,PathAnnotationObject,Polygon,455.32,163.11,9.4114,12.8552,0.7157,...,71.1134,4.5568,121.9538,543.2628,13.91,-187.23,187.746001,75_17,75,17


## Write abdomen and hemocyte features to csv files for downstream analysis 

In [12]:
hemocyte_coordinates.to_csv(outputs.joinpath("hemocyte_coordinates_midsections.csv"), sep="\t",index=False)
abdomen_coordinates.to_csv(outputs.joinpath("abdomen_coordinates_midsections.csv"), sep="\t",index=False)